In [49]:
import os
import datetime
from datetime import timedelta
from dateutil.relativedelta import relativedelta
import plotly.express as px
import plotly

In [50]:
print('Inicializado Rotina json_collect.py.')
os.chdir(sys.path[0])
# os.chdir(os.path.join(os.getcwd(),"json_collect.py"))                  # Modifica o diretório de trabalho, para os caminhos relativos funcionar. (https://stackoverflow.com/questions/1432924/python-change-the-scripts-working-directory-to-the-scripts-own-directory)
# print(os.getcwd())
%run ./json_collect.py                      


Inicializado Rotina json_collect.py.


In [51]:
# sites: https://dadosabertos.bcb.gov.br/dataset/expectativas-mercado
# Sintase: https://dadosabertos.bcb.gov.br/dataset/expectativas-mercado/resource/d420a704-75a7-4f45-8f4b-0fca813c70f0
# documentação: https://olinda.bcb.gov.br/olinda/servico/Expectativas/versao/v1/documentacao

In [52]:
df_selic_data=pd.read_csv('./input/selic_datas.csv',index_col='Reuniao')
df_selic=pd.read_csv('./output/selic_data_df.csv',index_col='Reuniao')
df_selic=df_selic.drop(columns=['Unnamed: 0','Indicador'])
df_selic['Data']=df_selic.apply(lambda x: datetime.datetime.strptime(x.Data,'%Y-%m-%d').date(),axis=1)

df_selic['data_reuniao']=''
for i in range(0, len(df_selic)):
    busca_data=''
    try:
        busca_data=df_selic_data.loc[df_selic.index[i],'data_reuniao']
        busca_data=datetime.datetime.strptime(busca_data,'%Y-%m-%d').date()
        df_selic.loc[df_selic.index[i],'data_reuniao']=busca_data

    except:
        # será definida um expectativa para a data.
        # busca_data=df_selic_data.loc[df_selic.index[i],'data_reuniao']
        # df_selic.loc[df_selic.index[i],'data_reuniao']=busca_data
        continue

# df_selic['data_reuniao']=df_selic.apply(lambda x: datetime.datetime.strptime(x.data_reuniao,'%Y-%m-%d').date(),axis=1)
# df_selic=df_selic[df_selic['data_reuniao']!='']
df_selic=df_selic.sort_values(by=['Data','data_reuniao','baseCalculo'],ascending=[False,True,False])
df_selic=df_selic[df_selic['baseCalculo']==1]           # não sei qual é a diferença da base de cálculo.
df_selic.head(20)

,Data,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo,data_reuniao
Reuniao,,,,,,,,,
R6/2022,2022-08-05,13.8869,13.7500,0.1562,13.75,14.25,42,1,2022-09-21
R7/2022,2022-08-05,13.8869,13.7500,0.1562,13.75,14.25,42,1,2022-10-26
R8/2022,2022-08-05,13.8810,13.7505,0.1658,13.50,14.25,42,1,2022-12-07
R1/2023,2022-08-05,13.8397,13.7500,0.2079,13.25,14.25,39,1,2023-02-01
R2/2023,2022-08-05,13.7564,13.7500,0.3276,12.75,14.25,39,1,2023-03-22
R3/2023,2022-08-05,13.5705,13.7500,0.5249,12.00,14.25,39,1,2023-05-03
R4/2023,2022-08-05,13.2885,13.5000,0.7458,11.50,14.25,39,1,2023-06-21
R5/2023,2022-08-05,12.7628,13.0000,0.8242,10.50,14.00,39,1,2023-08-02
R6/2023,2022-08-05,12.2308,12.5000,0.8518,10.25,14.00,39,1,2023-09-20


In [53]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_selic,x='data_reuniao',y='Mediana', color='Data')
# Gerando arquivo com o resultado
# plotly.offline.plot(fig, show_link = True,filename=f'./output/IPCA_forcast.html')
fig.show()

In [77]:
a=df_selic_ultima.iloc[0,-3]-datetime.datetime.today().date()
a.days

43

In [65]:
datetime.datetime.today().date()

datetime.date(2022, 8, 9)

In [81]:
# Expectativa mais atual

df_selic_ultima=df_selic[df_selic['Data']==df_selic['Data'].max()]
df_selic_ultima['mediana_mensal']=df_selic_ultima.apply(lambda x: (1+(x.Mediana/100))**(1/12)-1,axis=1)
df_selic_ultima['num_indice']=1000
pd.options.display.float_format = '{:,.2f}'.format
#### Definir número indice  #######
# df_selic_ultima['num_indice']=df_selic_ultima.apply(lambda x: x.num_indice*((1+x.Mediana/100)**((x.data_reuniao-datetime.datetime.today().date()).days)/(365)),axis=1)  
# df_selic_ultima['cumsum_mediana_mensal']=df_selic_ultima['mediana_mensal'].cumsum()
df_selic_ultima.to_csv('./output/selic_ultima_expectativa.csv')
df_selic_ultima

C:\Users\bruno.bfn\AppData\Local\Temp\ipykernel_3940\3957757982.py:4: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\bruno.bfn\AppData\Local\Temp\ipykernel_3940\3957757982.py:5: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

C:\Users\bruno.bfn\AppData\Local\Temp\ipykernel_3940\3957757982.py:9: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.

,Data,Media,Mediana,DesvioPadrao,Minimo,Maximo,numeroRespondentes,baseCalculo,data_reuniao,mediana_mensal,num_indice
Reuniao,,,,,,,,,,,
R6/2022,2022-08-05,13.89,13.75,0.16,13.75,14.25,42,1,2022-09-21,0.01,697.62
R7/2022,2022-08-05,13.89,13.75,0.16,13.75,14.25,42,1,2022-10-26,0.01,"63,374.82"
R8/2022,2022-08-05,13.88,13.75,0.17,13.50,14.25,42,1,2022-12-07,0.01,"14,194,001.70"
R1/2023,2022-08-05,13.84,13.75,0.21,13.25,14.25,39,1,2023-02-01,0.01,"19,282,106,332.30"
R2/2023,2022-08-05,13.76,13.75,0.33,12.75,14.25,39,1,2023-03-22,0.01,"10,635,877,694,767.55"
R3/2023,2022-08-05,13.57,13.75,0.52,12.00,14.25,39,1,2023-05-03,0.01,"2,380,851,775,594,127.00"
R4/2023,2022-08-05,13.29,13.50,0.75,11.50,14.25,39,1,2023-06-21,0.01,"655,238,456,712,547,200.00"
R5/2023,2022-08-05,12.76,13.00,0.82,10.50,14.00,39,1,2023-08-02,0.01,"27,528,966,202,760,794,112.00"
R6/2023,2022-08-05,12.23,12.50,0.85,10.25,14.00,39,1,2023-09-20,0.01,"1,806,274,586,470,510,231,552.00"


In [55]:
df_monthly=pd.read_csv('./output/monthly_data_df.csv')
df_monthly=df_monthly.drop(columns=['Unnamed: 0'])

# Parses
df_monthly['Data']=df_monthly.apply(lambda x: datetime.datetime.strptime(x.Data,'%Y-%m-%d').date(),axis=1)

df_monthly['DataReferencia']=df_monthly.apply(lambda x: datetime.datetime.strptime(x.DataReferencia,'%m/%Y').date(),axis=1)
delta = relativedelta(months=1)
delta_d = timedelta(days=1)
df_monthly['DataReferencia']=df_monthly.apply(lambda x: x.DataReferencia+delta-delta_d,axis=1)

df_monthly=df_monthly.sort_values(by=['Data','DataReferencia','numeroRespondentes'],ascending=[False,True,False])
# df_monthly=df_selic[df_selic['baseCalculo']==1]           # não sei qual é a diferença da base de cálculo.
df_monthly.head(20)

,Indicador,Data,DataReferencia,Media,Mediana,Minimo,Maximo,numeroRespondentes
407,IPCA,2022-08-05,2022-07-31,-0.6148,-0.640,-1.0600,0.6200,138
419,IPCA Administrados,2022-08-05,2022-07-31,-4.1348,-4.260,-4.8700,-1.2000,79
411,IPCA Livres,2022-08-05,2022-07-31,0.6140,0.680,-0.0250,0.8000,73
417,IPCA Alimentação no domicílio,2022-08-05,2022-07-31,1.2985,1.355,0.0000,2.0500,68
413,IPCA Serviços,2022-08-05,2022-07-31,0.6316,0.730,-0.3590,0.9100,67
415,IPCA Bens industrializados,2022-08-05,2022-07-31,0.2313,0.200,-0.1600,1.4730,67
409,Taxa de desocupação,2022-08-05,2022-07-31,9.2751,9.200,8.6000,11.0000,53
406,IPCA,2022-08-05,2022-07-31,-0.6034,-0.635,-0.7584,0.0784,48
418,IPCA Administrados,2022-08-05,2022-07-31,-4.1286,-4.260,-4.7249,-1.2000,22
410,IPCA Livres,2022-08-05,2022-07-31,0.6340,0.670,-0.0250,0.7514,19


In [56]:
df_monthly['Indicador'].unique()

array(['IPCA', 'IPCA Administrados', 'IPCA Livres',
       'IPCA Alimentação no domicílio', 'IPCA Serviços',
       'IPCA Bens industrializados', 'Taxa de desocupação', 'Câmbio',
       'IGP-M'], dtype=object)

In [57]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_monthly[df_monthly['Indicador']=='IPCA'],x='DataReferencia',y='Mediana', color='Data')
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/IPCA_forcast.html')
# fig.show()

# Gráfico iterativo utilizando o plotly
fig=px.line(df_monthly[df_monthly['Indicador']=='IGP-M'],x='DataReferencia',y='Mediana', color='Data')
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/IGPM_forcast.html')
# fig.show()

# Gráfico iterativo utilizando o plotly
fig=px.line(df_monthly[df_monthly['Indicador']=='Câmbio'],x='DataReferencia',y='Mediana', color='Data')
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/cambio_forcast.html')
# fig.show()

# Gráfico iterativo utilizando o plotly
fig=px.line(df_monthly[df_monthly['Indicador']=='Taxa de desocupação'],x='DataReferencia',y='Mediana', color='Data')
# Gerando arquivo com o resultado
plotly.offline.plot(fig, show_link = True,filename=f'./output/tx_desocupacao_forcast.html')
# fig.show()

'./output/tx_desocupacao_forcast.html'

In [58]:
# Expectativa mais atual para IPCA
df_ipca=df_monthly[df_monthly['Indicador']=='IPCA']

data_previsao=df_ipca['Data'].max()
df_ipca_ultima=df_ipca[df_ipca['Data']==data_previsao]
df_ipca_ultima.to_csv('./output/ipca_ultima_expectativa.csv')
df_ipca_ultima=df_ipca_ultima.groupby(['DataReferencia']).mean() # adotado média 

df_ipca_ultima['cumsum_mediana'] = df_ipca_ultima['Mediana'].cumsum()
#cumsum()-> soma acumulada de uma lista ou array termo a termo
#groupby -> delimita em grupos separados para análise
# df['Adjusted Quantity'] = df.groupby('ação_ref')['Adjusted Quantity'].cumsum()

df_ipca_ultima

,Media,Mediana,Minimo,Maximo,numeroRespondentes,cumsum_mediana
DataReferencia,,,,,,
2022-07-31,-0.60910,-0.63750,-0.90920,0.34920,93.0,-0.63750
2022-08-31,-0.13695,-0.15000,-0.57960,0.45000,93.0,-0.78750
2022-09-30,0.48935,0.50000,0.13600,0.71000,93.0,-0.28750
2022-10-31,0.55110,0.53745,0.24510,0.84000,92.0,0.24995
2022-11-30,0.53610,0.53525,0.15000,0.79000,92.0,0.78520
2022-12-31,0.73280,0.74775,0.10040,1.14500,92.0,1.53295
2023-01-31,0.78405,0.69500,0.26900,1.49500,89.5,2.22795
2023-02-28,0.66630,0.67500,0.22500,1.17000,89.5,2.90295
2023-03-31,0.48960,0.48000,0.02500,1.31280,89.5,3.38295


In [59]:
# from matplotlib.pyplot import figure
# figure(figsize=(10, 5), dpi=120)
# plt.plot(df_ipca_ultima.index, df_ipca_ultima['Mediana'], label=f'ipca_expectaiva_{data_previsao}')
# plt.xticks(rotation=90)
# plt.legend()
# plt.show()


In [60]:
data_hoje=datetime.datetime.today().date()
df_ipca_from_now=df_ipca_ultima[df_ipca_ultima.index>=data_hoje]
df_ipca_from_now=df_ipca_from_now.reset_index()
df_ipca_from_now

,DataReferencia,Media,Mediana,Minimo,Maximo,numeroRespondentes,cumsum_mediana
0,2022-08-31,-0.13695,-0.15000,-0.57960,0.45000,93.0,-0.78750
1,2022-09-30,0.48935,0.50000,0.13600,0.71000,93.0,-0.28750
2,2022-10-31,0.55110,0.53745,0.24510,0.84000,92.0,0.24995
3,2022-11-30,0.53610,0.53525,0.15000,0.79000,92.0,0.78520
4,2022-12-31,0.73280,0.74775,0.10040,1.14500,92.0,1.53295
5,2023-01-31,0.78405,0.69500,0.26900,1.49500,89.5,2.22795
6,2023-02-28,0.66630,0.67500,0.22500,1.17000,89.5,2.90295
7,2023-03-31,0.48960,0.48000,0.02500,1.31280,89.5,3.38295
8,2023-04-30,0.45255,0.44500,0.11000,0.79000,89.5,3.82795
9,2023-05-31,0.33155,0.34295,0.03000,0.72200,89.5,4.17090


In [61]:
# Gráfico iterativo utilizando o plotly
fig=px.line(df_ipca_from_now,x='DataReferencia',y='cumsum_mediana')
# Gerando arquivo com o resultado
# plotly.offline.plot(fig, show_link = True,filename=f'./output/tx_desocupacao_forcast.html')
fig.show()